<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/RAG/Memory_in_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing necessary libraries

Cohere provides free trial keys to use their LLMs. So generate one trial key from dashboard.cohere.com

In [1]:
!pip install langchain-cohere langchain pdfminer.six chromadb pydantic==1.10.8 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.2/250.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00

langchain-cohere: Enables integration of Cohere's language models with LangChain for advanced text generation and processing workflows.

langchain: Provides a modular framework for building language model-powered applications, such as chatbots, question-answering systems, and conversational agents.

pdfminer.six: Facilitates text extraction from PDF files, making it useful for document analysis and preprocessing tasks.

chromadb: A vector database library designed for efficient storage and retrieval of embeddings, ideal for tasks like semantic search and recommendation systems.

## Importing libraries

In [2]:
import os
from typing import List
from pydantic import BaseModel, Field
from langchain_core.messages import BaseMessage, AIMessage,HumanMessage
from google.colab import userdata
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_KEY')
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_cohere import ChatCohere
from langchain.schema.output_parser import StrOutputParser
from pdfminer.high_level import extract_text as extract_text_pdf_miner
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain

## VectorDB setup

In [4]:
# Define the directory where the Chroma database will persist data
persist_directory = "/content/chroma_db"

# Initialize Cohere embeddings with the specified model
# "embed-english-v3.0" is a pre-trained English language embedding model by Cohere
# The user_agent parameter specifies the tool or library using the Cohere API, in this case, LangChain
embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)

We are processing 2 research papers on ai agents and genai on vertex ai. You can use the PDFs

In [5]:
# Loop through a list of PDF files to process
for pdf_name in ["/content/Newwhitepaper_Agents.pdf", "/content/Newwhitepaper_Operationalizing Generative AI on Vertex AI.pdf"]:
    # Open each PDF file in binary mode
    with open(pdf_name, 'rb') as f:
        # Extract text from the PDF using the extract_text_pdf_miner function
        text = extract_text_pdf_miner(f)

        # Clean the extracted text by removing newline characters and joining into a single string
        cleaned_text = " ".join(text.split("\n"))

        # Initialize a list to store document chunks
        docs = []

        # Create a text splitter to divide the text into manageable chunks
        # Each chunk has a maximum size of 2048 characters with a 512-character overlap
        splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

        # Split the cleaned text into chunks and wrap each chunk in a Document object
        for chunk in splitter.split_text(cleaned_text):
            docs.append(Document(page_content=chunk, metadata={"source": pdf_name}))

    # Create a Chroma collection from the processed documents
    # Use the specified persist directory and embedding model for storage and retrieval
    vector_collection_fixed_size = Chroma.from_documents(
        documents=docs,
        persist_directory=persist_directory,
        embedding=embedding
    )

In [6]:
# Initialize a Chroma vector database
# The persist_directory specifies the location where the database is stored
# The embedding_function parameter provides the embedding model used for vector representation
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

<ipython-input-6-66dc66a8b09c>:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [ ]:
# Perform a similarity search on the vector database
# The query "What is YOLO?" is used to find the most relevant documents
# k=1 specifies that the top 1 most similar document should be retrieved
# The method also returns relevance scores indicating how closely each document matches the query
vectordb.similarity_search_with_relevance_scores("What is YOLO?", k=1)

[(Document(metadata={'source': '/content/1506.02640v5.pdf'}, page_content='Detection In The Wild  Academic datasets for object detection draw the training and testing data from the same distribution. In real-world applications it is hard to predict all possible use cases and  YOLO is a fast, accurate object detector, making it ideal for computer vision applications. We connect YOLO to a webcam and verify that it maintains real-time performance,  \x0cVOC 2007 AP 59.2 54.2 43.2 36.5 -  Picasso AP Best F1 0.590 53.3 0.226 10.4 0.458 37.8 0.271 17.8 0.051 1.9  People-Art AP 45 26 32  YOLO R-CNN DPM Poselets [2] D&T [4]  (a) Picasso Dataset precision-recall curves.  (b) Quantitative results on the VOC 2007, Picasso, and People-Art Datasets. The Picasso Dataset evaluates on both AP and best F1 score.  Figure 5: Generalization results on Picasso and People-Art datasets.  Figure 6: Qualitative Results. YOLO running on sample artwork and natural images from the internet. It is mostly accurate a

## Chain Setup

In [ ]:
# Initialize an LLM instance using Cohere's "command-r" model
# The temperature parameter controls randomness in the generated responses; 0 ensures deterministic outputs
llm = ChatCohere(model="command-r", temperature=0)

In [ ]:
# Define a prompt template for generating answers based on a given context and question
prompt_str = """Given a chat history and the latest user question which might reference context in the chat history,
formulate a standalone question which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

# Use a prompt that includes a MessagesPlaceholder variable under the name "chat_history".
# This allows us to pass in a list of Messages to the prompt using the "chat_history" input key,
# and these messages will be inserted after the system message and before the human message containing the
# latest question.

prompt_history_aware = ChatPromptTemplate.from_messages([
    ("system", prompt_str),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# create_history_aware_retriever constructs a chain that accepts keys input and chat_history as input, and has the same output schema as a retriever
history_aware_retriever = create_history_aware_retriever(
    llm, vectordb.as_retriever(), prompt_history_aware
)

Now its time to build our final rag_chain with create_retrieval_chain. This chain applies the history_aware_retriever and question_answer_chain created with create_stuff_documents_chain in sequence, retaining intermediate outputs such as the retrieved context for convenience. It has input keys input and chat_history, and includes input, chat_history, context, and answer in its output.



In [ ]:
system_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question. If you don't know the answer,
say that you don't know. Use three sentences maximum and keep theanswer concise.

{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# create_retrieval_chain combines history aware retriever and the qa chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
# Load a empty chat_history list
chat_history = []

# Invoking the chain
question = "What is YOLO?"
response = rag_chain.invoke({"input": question, "chat_history": chat_history})

# Appeding question and response answers
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response["answer"]),
    ]
)

In [ ]:
#check chat_history
chat_history

[HumanMessage(content='What is YOLO?'),
 AIMessage(content='YOLO is a real-time object detection system, which uses a single convolutional neural network to simultaneously predict multiple bounding boxes and class probabilities for those boxes. The name YOLO is an acronym for "You Only Look Once".')]

In [ ]:
second_question = "What are the tasks YOLO can be used in?"
response = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(response["answer"])

YOLO is a fast and accurate object detector which can be applied to various tasks including person detection and tracking objects in real-time. It's also effective for use in computer vision applications and general purpose robotic systems due to its real-time performance.


To automate the inserting and updating of chat history. And have a session id that can be unique for a user

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is Transformers?"},
    config={
        "configurable": {"session_id": "User_1"}
    },
)["answer"]

"Transformers are a type of neural network architecture that excels in sequence transduction tasks, such as machine translation and language modeling. They're called Transformers because they use self-attention mechanisms to transform input sequences into output sequences, hence the name. This architecture, first proposed in 2017, relies entirely on attention mechanisms and does not use recurrence or convolutions, which sets it apart from previous models."

In [ ]:
# This will output input, chat_history, contexts and answer
conversational_rag_chain.invoke(
    {"input": "Transformers vs YOLO"},
    config={
        "configurable": {"session_id": "User_1"}
    },
)

{'input': 'Transformers vs YOLO',
 'chat_history': [HumanMessage(content='What is Transformers?'),
  AIMessage(content="Transformers are a type of neural network architecture that excels in sequence transduction tasks, such as machine translation and language modeling. They're called Transformers because they use self-attention mechanisms to transform input sequences into output sequences, hence the name. This architecture, first proposed in 2017, relies entirely on attention mechanisms and does not use recurrence or convolutions, which sets it apart from previous models.")],
 'context': [Document(metadata={'source': '/content/1506.02640v5.pdf'}, page_content='You Only Look Once: Uniﬁed, Real-Time Object Detection  Joseph Redmon∗, Santosh Divvala∗†, Ross Girshick¶, Ali Farhadi∗† University of Washington∗, Allen Institute for AI†, Facebook AI Research¶ http://pjreddie.com/yolo/  6 1 0 2  y a M 9  ]  V C . s c [  5 v 0 4 6 2 0 . 6 0 5 1 : v i X r a  Abstract  We present YOLO, a new appro

In [ ]:
# Check the chat_history of the store dict
for message in store["User_1"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Transformers?

AI: Transformers are a type of neural network architecture that excels in sequence transduction tasks, such as machine translation and language modeling. They're called Transformers because they use self-attention mechanisms to transform input sequences into output sequences, hence the name. This architecture, first proposed in 2017, relies entirely on attention mechanisms and does not use recurrence or convolutions, which sets it apart from previous models.

User: Transformers vs YOLO

AI: YOLO (You Only Look Once) is a real-time object detection system that uses a single convolutional neural network to predict bounding boxes and class probabilities directly from full images. It is a fast and accurate object detector that processes images in real-time.

Transformers are a type of neural network architecture that excel in understanding long-range dependencies, which makes them perform well on tasks like machine translation and text summarization. They use s